<a href="https://colab.research.google.com/github/ShoaibMuhammad123/Pytorch/blob/main/Pytorch12_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset,DataLoader
import  torch.nn as nn
import torch.optim as optim

In [ ]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
device

'cuda'

In [ ]:
df = pd.read_csv('/content/fashion-mnist_train.csv')
df.head(3)

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0


In [ ]:
df.shape

(60000, 785)

In [ ]:
x = df.iloc[:,1:].values
y = df.iloc[:,0].values

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=.2,random_state=42)

In [ ]:
# transformations

# **Transformations**
* We have to define some transformation so, that out image become compatible for vgg16(which has trained on imgesnet data)

In [ ]:
# Now we define transformation

from torchvision.transforms import transforms

# inside compose ftn you can send any number of transformation inside a list
custom_transform = transforms.Compose([
    # 1- resize  transformation
    transforms.Resize(256),
    # 2-center crop
    transforms.CenterCrop(244),
    # 3-converting to tensor
    transforms.ToTensor(), # it convert into tensor as well as scalled the values
    # Normalize
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])

])

In [ ]:
a = np.array([[1,3],[4,5]])
a

array([[1, 3],
       [4, 5]])

In [ ]:
np.stack([a]*3)

array([[[1, 3],
        [4, 5]],

       [[1, 3],
        [4, 5]],

       [[1, 3],
        [4, 5]]])

In [ ]:
np.stack([a]*3).shape

(3, 2, 2)

In [ ]:
np.stack([a]*3,axis=1).shape

(2, 3, 2)

# **Custom Dataset**

In [ ]:
from PIL import Image
import numpy as np

In [ ]:
class CustomDataset(Dataset):

  def __init__(self,features,labels,transform):
    self.features = features
    self.labels = labels
    self.transform = transform

  def __len__(self):
    return len(self.features)

  def __getitem__(self,index):

    # resize (28,28)
    image = self.features[index].reshape(28,28)

    # change the type into np.uint8
    image = image.astype(np.uint8)

    # change black&white into colored --> (H,W,C)input  --> (C,H,W)output
    image = np.stack([image]*3,axis=-1)  # copying the image three times for making it RGB

    # converting array into PIL image
    image = Image.fromarray(image)

    # apply transforms
    image = self.transform(image)

    # return
    return image, torch.tensor(self.labels[index],dtype = torch.long)


In [ ]:
train_data = CustomDataset(x_train,y_train,transform=custom_transform)
test_data = CustomDataset(x_test,y_test,transform=custom_transform)

In [ ]:
# data loader
train_loader = DataLoader(train_data,batch_size=32,shuffle=True,pin_memory=True)
test_loader = DataLoader(test_data,batch_size=32,shuffle=True,pin_memory=True)

## Don't need to make your own model (**Becoz you will fetch the pretrained model**) from torchvision library

In [ ]:
import torchvision.models as models

vgg16 = models.vgg16(pretrained = True)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:07<00:00, 71.9MB/s]


In [ ]:
# now we check how the model looks
vgg16

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
vgg16.features

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): Conv2d(256, 512, kernel_si

In [ ]:
vgg16.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)

In [ ]:
# now First we freeze the features on vgg16
for param in vgg16.features.parameters():
  param.requires_grad = False    # it will not calculate the gradient on the features

In [ ]:
# now we replace the classifier part with our own classifier

# we define our own classifer and assign it to vgg16 classifier
vgg16.classifier= nn.Sequential(
    nn.Linear(25088,1024),
    nn.ReLU(),
    nn.Dropout(0.5),

    nn.Linear(1024,512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512,10)
)

In [ ]:
vgg16.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=1024, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=1024, out_features=512, bias=True)
  (4): ReLU()
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=512, out_features=10, bias=True)
)

In [ ]:
# sending it to gpu
vgg16 = vgg16.to(device)

In [ ]:
learning_rate = 0.0001
epochs = 5

In [ ]:
# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vgg16.classifier.parameters(),lr = learning_rate)

# **Training Loop**

In [ ]:
for epoch in range(epochs):
  total_epoch_loss = 0

  for batch_features, batch_labels in train_loader:

    # batch_features and labels into gpu
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    # forward pass
    outputs = vgg16(batch_features)

    # loss
    loss = criterion(outputs,batch_labels)

    # backward pass
    optimizer.zero_grad()
    loss.backward()

    # update parameter
    optimizer.step()

    total_epoch_loss = total_epoch_loss + loss.item()

  avg_loss = total_epoch_loss/len(train_loader)

  print(f'Epoch {epoch} and Loss {avg_loss}')







Epoch 0 and Loss 0.30078761564195156
Epoch 1 and Loss 0.20579570640188952
Epoch 2 and Loss 0.154635007928436
Epoch 3 and Loss 0.12165337663469836
Epoch 4 and Loss 0.09368020016102431


# **Evaluation**

In [ ]:
# Evaluation loop for test data
vgg16.eval()

total=0
correct =0

with torch.no_grad():

  for batch_features, batch_labels in test_loader:
     batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

     ouputs = vgg16(batch_features)

     _,predicted = torch.max(outputs ,1)

     total = total + batch_labels.shape[0]

     correct = correct + (predicted == batch_labels).sum().item()
print(correct /total)




0.10383333333333333


In [ ]:
# Evaluation loop for train data
vgg16.eval()

total=0
correct =0

with torch.no_grad():

  for batch_features, batch_labels in train_loader:
     batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

     ouputs = vgg16(batch_features)

     _,predicted = torch.max(outputs ,1)

     total = total + batch_labels.shape[0]

     correct = correct + (predicted == batch_labels).sum().item()
print(correct /total)

0.10254166666666667
